In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch import nn,optim
from torch.nn import functional as F 
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
traindata = np.load('2ddata.npy')
trainlabel = np.load('2dlabel.npy')

In [4]:
trainlabel

array([ 43.52606322, -13.99713994,   9.49569717, ..., -12.22158433,
         6.01721102, -24.02406788])

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(traindata, trainlabel, test_size = 0.2, random_state = 1)
X_train, Y_train = torch.FloatTensor(X_train), torch.FloatTensor(Y_train)
X_test, Y_test = torch.FloatTensor(X_test), torch.FloatTensor(Y_test)

In [6]:
batchsize = 8
train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, Y_test)

#dataloader 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batchsize, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batchsize, shuffle = False)

In [7]:
X_train.shape 

torch.Size([16000, 32, 32])

In [30]:
class CNN_16d2(nn.Module):
    def __init__(self):
        super(CNN_16d2,self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 2,  stride=2)
        self.conv2 = nn.Conv2d(4, 16, 2, stride=2)
        self.conv3 = nn.Conv2d(16, 160, 2, stride=2)
        self.conv4 = nn.Conv2d(160, 80, 2, stride=2)
        self.conv5 = nn.Conv2d(80, 20, 2, stride=2)

        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(20, 60)
        self.fc2 = nn.Linear(60, 60)
        self.fc3 = nn.Linear(60, 20)
        self.fc4 = nn.Linear(20, 1)
    
    def forward(self,x):
        x = self.conv1(x)
        #print(x.shape)
        x = self.conv2(x)
        #print(x.shape)
        x = self.conv3(x)
        #print(x.shape)
        x = self.conv4(x)  
        #print(x.shape)
        x = self.conv5(x)  
        #print(x.shape)

        x = self.flat(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [31]:
X = torch.randn(1,1,32,32)
xiao = CNN_16d2()
xiao(X)

tensor([[-0.0230]], grad_fn=<AddmmBackward0>)

In [32]:
model = CNN_16d2()
if torch.cuda.is_available():
    print("gpu_available")
    model.cuda()

gpu_available


In [33]:
optimizer = optim.Adam(model.parameters(), lr =0.001)
criteon= nn.MSELoss()
EPOCH = 4
loss_list = []
for epoch in range(EPOCH):
    model.train()
    sum_loss = 0 
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        #data, target = Variable(data), Variable(target)

        data = torch.reshape(data,(batchsize, 1, 32,32))
        target = torch.reshape(target, (batchsize, 1))
        optimizer.zero_grad()
        
        
        data=data.to(torch.float32)
        target=target.to(torch.float32)
        output = model(data)

        
        #print(data)
        #print(target)
        #print("The output is:")
        #print(output)
        
        loss = criteon(output, target)
        loss.backward()
        optimizer.step()

        sum_loss += loss.item()
        if batch_idx % 10 == 9:
            print("train epoch {} loss : {:.3f} ".format(epoch, sum_loss/10))
            loss_list.append(sum_loss/100)
            sum_loss = 0      

train epoch 0 loss : 1252.458 
train epoch 0 loss : 1211.436 
train epoch 0 loss : 1628.970 
train epoch 0 loss : 2255.892 
train epoch 0 loss : 2212.110 
train epoch 0 loss : 2488.308 
train epoch 0 loss : 1174.659 
train epoch 0 loss : 1566.575 
train epoch 0 loss : 1287.551 
train epoch 0 loss : 1673.098 
train epoch 0 loss : 1559.372 
train epoch 0 loss : 1128.815 
train epoch 0 loss : 1483.951 
train epoch 0 loss : 1310.568 
train epoch 0 loss : 1177.141 
train epoch 0 loss : 1125.729 
train epoch 0 loss : 1442.870 
train epoch 0 loss : 1510.841 
train epoch 0 loss : 1704.030 
train epoch 0 loss : 1365.248 
train epoch 0 loss : 1499.358 
train epoch 0 loss : 1601.261 
train epoch 0 loss : 1569.443 
train epoch 0 loss : 1811.768 
train epoch 0 loss : 1483.493 
train epoch 0 loss : 1312.738 
train epoch 0 loss : 1018.830 
train epoch 0 loss : 1555.806 
train epoch 0 loss : 1308.251 
train epoch 0 loss : 1179.168 
train epoch 0 loss : 1473.892 
train epoch 0 loss : 1655.182 
train ep

In [34]:
model = model.cpu()
re = model(torch.Tensor(X_test.reshape(4000,1,32,32))).detach().numpy()
re = re.reshape(4000)
re

array([ 55.43486  ,   5.557933 ,  49.076565 , ...,   5.5215306,
       -36.24969  ,  -3.6554797], dtype=float32)

In [35]:
Y_test

tensor([ 61.5076,   4.2478,  48.3385,  ...,   3.4108, -27.6324,  -7.8302])

In [36]:
resultvalue = pd.DataFrame([re,Y_test.numpy()]).T
resultvalue.columns = ['pre','true']
resultvalue

,pre,true
0,55.434860,61.507622
1,5.557933,4.247818
2,49.076565,48.338482
3,25.994368,23.041401
4,55.268932,68.316994
...,...,...
3995,5.268528,-6.869918
3996,6.880189,4.799513
3997,5.521531,3.410830
3998,-36.249691,-27.632389


In [37]:
resultvalue['mape'] = np.abs(resultvalue['pre'] - resultvalue['true'])/np.abs(resultvalue['true'])
resultvalue

,pre,true,mape
0,55.434860,61.507622,0.098732
1,5.557933,4.247818,0.308421
2,49.076565,48.338482,0.015269
3,25.994368,23.041401,0.128159
4,55.268932,68.316994,0.190993
...,...,...,...
3995,5.268528,-6.869918,1.766898
3996,6.880189,4.799513,0.433518
3997,5.521531,3.410830,0.618823
3998,-36.249691,-27.632389,0.311855


In [41]:
resultvalue['mape'].median()

0.17648991546688447

In [25]:
resultvalue.to_csv("hhh.csv")

In [39]:
torch.save(model, '2d32-80-20model.pt')